# Pymongo Tutorial

This notebook goes through a quick, easy example of how to use **pymongo**, a python library, to work with **MongoDB**, a NoSQL database program.

If you don't already have MongoDB installed on your system (assuming mac/linux operating system) follow the directions found [here](https://docs.mongodb.com/manual/tutorial/install-mongodb-on-ubuntu/) or type the following commands into the terminal:

1. `sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv 0C49F3730359A14518585931BC711F9BA15703C6`
2. `echo "deb [ arch=amd64,arm64 ] http://repo.mongodb.org/apt/ubuntu xenial/mongodb-org/3.4 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-3.4.list`

3. `sudo apt-get update`
4. `sudo apt-get install -y mongodb-org`

In [1]:
# if pymongo not installed on system, run 'sudo pip install pymongo' from the terminal
from pymongo import MongoClient

In [2]:
# get our MongoClient object
client = MongoClient()

The mongo [daemon process](https://docs.mongodb.com/manual/reference/program/mongod/) must be running in order for you to access your databases. Unless you set up your system so that the daemon runs in the background (see [here](https://serverfault.com/questions/157705/how-can-i-run-mongod-in-the-background-on-unix-mac-osx) for a discussion of how to do this), you'll need to run `sudo mongod` in the terminal, which will occupy that terminal tab while the daemon is running.

If the daemon isn't running, you'll most likely get a `ServerSelectionTimeoutError` when you run the code immediately below.

In [3]:
# list existing databases, 'admin' and 'local' are automatically created when MongoDB is installed
client.database_names()

['admin', 'local']

In [4]:
# MongoDB will automatically create a database if it doesn't already exist,
# so the below code can be used to access an existing database or create a new one
test_db = client.get_database('test')

In [5]:
# list existing collections within a database
test_db.collection_names()

[]

In [6]:
# As with databases, mongoDB will automatically create a collection if it didn't already exist
test_collection = test_db['test_collection']

In [7]:
# our sample documents to insert into the test_collection
dictionary_items = []

dictionary_items.append({'name': 'bandit', 'type': 'cat', 'age': 7, 'friendly': False})
dictionary_items.append({'name': 'alfred', 'type': 'dog', 'age': 5, 'friendly': True })
dictionary_items.append({'name': 'lillie', 'type': 'cat', 'age': 9, 'friendly': True })

In [8]:
# insert our documents into the test_collection

for item in dictionary_items:
    test_collection.insert_one(item)

In [9]:
# how many documents are in our collection?
test_collection.count()

3

In [10]:
# now that there are documents in our new collection in our new database,
# both the collection and database will show up when we print our databases and collections

print(client.database_names())
print(test_db.collection_names())

['admin', 'local', 'test']
['test_collection']


In general, the easiest way to go through a collection is to generate a cursor object with `find()` and iterate through each document.

In [11]:
for doc in test_collection.find():
    print(doc)

{'_id': ObjectId('59a347aaacef4a062aaf2997'), 'name': 'bandit', 'type': 'cat', 'age': 7, 'friendly': False}
{'_id': ObjectId('59a347aaacef4a062aaf2998'), 'name': 'alfred', 'type': 'dog', 'age': 5, 'friendly': True}
{'_id': ObjectId('59a347aaacef4a062aaf2999'), 'name': 'lillie', 'type': 'cat', 'age': 9, 'friendly': True}


In [12]:
for doc in test_collection.find():
    print('Name: ', doc['name'], ', Age: ', doc['age'], sep='')

Name: bandit, Age: 7
Name: alfred, Age: 5
Name: lillie, Age: 9


In [13]:
# to add an additional key to a document, we use find() to iterate through, and filter using _id,
# which is an automatically generated unique id for each document.
# update_one() allows us to update the document, and $set assigns the new key and value.
# you can also update an existing value with this same method (not shown here)

for doc in test_collection.find():
    if doc['friendly']:
        test_collection.update_one({'_id': doc['_id']}, {'$set': {'good_with_kids': True}})
    else:
        test_collection.update_one({'_id': doc['_id']}, {'$set': {'good_with_kids': False}})

In [14]:
for doc in test_collection.find():
    print(doc)

{'_id': ObjectId('59a347aaacef4a062aaf2997'), 'name': 'bandit', 'type': 'cat', 'age': 7, 'friendly': False, 'good_with_kids': False}
{'_id': ObjectId('59a347aaacef4a062aaf2998'), 'name': 'alfred', 'type': 'dog', 'age': 5, 'friendly': True, 'good_with_kids': True}
{'_id': ObjectId('59a347aaacef4a062aaf2999'), 'name': 'lillie', 'type': 'cat', 'age': 9, 'friendly': True, 'good_with_kids': True}


In [15]:
# we can easily delete documents based on key values
# you can also assign this result to a variable if you need to reference the deleted items

test_collection.delete_many({'friendly': True})

In [16]:
for doc in test_collection.find():
    print(doc)

{'_id': ObjectId('59a347aaacef4a062aaf2997'), 'name': 'bandit', 'type': 'cat', 'age': 7, 'friendly': False, 'good_with_kids': False}


In [17]:
# if you need to delete multiple documents based on more complicated criteria, iterate through like we did for updating

for doc in test_collection.find():
    if doc['age'] <= 7:
        test_collection.delete_one({'_id': doc['_id']})

In [18]:
test_collection.count()

0

In [19]:
# to get rid of a collection, we use drop_collection()

test_db.drop_collection('test_collection')

{'nIndexesWas': 1, 'ns': 'test.test_collection', 'ok': 1.0}

In [20]:
# to get rid of a database, we use drop_database()

client.drop_database('test')